# Resource planning

In [16]:
%%html
<style>
table {float:left}
</style>

| Document info | |
| --- | --- | 
| Area of interest: | AMK-TP Tender |
| Planning type: | All REL type producers |
| Prepared by: | Waste Labs (wastelabs.co) |
| Prepared for: | ALBW WH |
| Contact: | elias@wastelabs.co |

In [19]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

# Show all code cells outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import logging
logging.basicConfig(level=logging.INFO)

import pickle
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
import plotly.express as px
init_notebook_mode(connected=True)

import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(colorscale='plotly', world_readable=True)

import os
import sys
#sys.path.insert(0, '../../../mcarptif/')
#sys.path.insert(0, os.path.abspath('../../collection_diagnostics/'))
#sys.path.insert(0, os.path.abspath('../../OSM_processing/'))
import utils.process_gdf as process_gdf

import plotly.graph_objects as go

from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import clear_output
import ipywidgets as widgets

import pandas as pd
import geopandas as gpd
from shapely import wkt
import numpy as np

from keplergl import KeplerGl
# Extra options
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000

from mcarptif.osmnx_network_extract.extract_grptif import NetworkExtract
from mcarptif.visualise.route_tables import RouteSummary
#from mcarptif.visualise.customer_plots import color_df
#from mcarptif.visualise.pydeck_routes import plot_routes
from mcarptif.solver.solve import solve_store_instance
from mcarptif.osmnx_network_olveextract.network_code import create_gdf
from mcarptif.osmnx_network_extract.network_code import required_arc_plot
from mcarptif.osmnx_network_extract.network_code import create_gdf
from utils import df_visual, df_prep, gdf_helpers

def df_style(df):
    if df[0] == 'Total':
        return ['font-weight: bold'] * len(df)
    else:
        return [''] * len(df)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


SyntaxError: invalid syntax (py_alg_ulusoy_partition.py, line 409)

In [ ]:
input_file = '../hobbes_projects/wasted_large_instances/outputs/city_punggol_full_hull_network_arcs_remaining'
network_file = '../../data_singapore/full_network_preperation/sp_files/SG_full_road_simple_edges.csv'
network_file_directed = '../../data_singapore/full_network_preperation/sp_files/SG_full_road_simple_edges_bidirect_clean.csv'
nodes_file = '../../data_singapore/full_network_preperation/sp_files/SG_full_road_simple_nodes.csv'
networkh5_file = "../data/05_model_input/sp_files/road_network_simplified_24645.h"

test_network = pd.read_csv(network_file, dtype={'index_right': str, 'type': str, 'parallel': str})
test_network = test_network.drop(columns=['Unnamed: 0'], errors='ignore')
test_network_directed = pd.read_csv(network_file_directed)

nodes = pd.read_csv(nodes_file)

In [ ]:
%%time
test_network = process_gdf.process_edges(test_network)
test_network.drop_duplicates(['geom_id'], inplace=True)
nodes.sp_index = nodes.index
test_network['arc_index'] = test_network.index
nodes['sp_index'] = nodes.index

## Service area

In [ ]:
input_directory = 'customer_allocation_final/'
input_files = os.listdir(input_directory)
input_files = [file for file in input_files if file[0] != '.']
input_files

filename=[
         'landed.csv']

outlist = filename
df_producer_geo = [pd.read_csv(input_directory + f, dtype={'Postal Code': str}) for f in filename]

outliers_landed = []#'357775', '357777', '357783', '357784', '357785', '357790', '357791', '357796', '357798', '358456', '358457']

df_producer_geo = pd.concat(df_producer_geo)
df_producer_geo['land_use'] = df_producer_geo['land_use'].fillna('nan')
df_producer_geo = df_producer_geo.merge(test_network[['arc_id', 'geom_id', 'geom_id_inv', 'geom_id_order', 'arc_index']])
df_producer_geo.loc[df_producer_geo['Postal Code'].isin(outliers_landed), 'category'] = 'Landed - outliers'
display('Producers in area: {}'.format(df_producer_geo.shape[0]))

## Add service time and demand

In [ ]:
routing_parameters = pd.read_excel('../routing_parameters/refuse_parameters.xlsx', sheet_name = 'Refuse_P1')
routing_parameters.fillna(0, inplace=True)
routing_parameters

In [ ]:
calculate = ['demand', 'bin_service_cost']
df_producer_geo.drop(columns=calculate, inplace=True, errors='ignore')
df_producer_geo['Domestic'] = 1
df_producer_geo['Non Domestic'] = 0
df_producer_geo['H/M'] = 0
df_producer_geo['category'] = 'Landed'
df_producer_geo['land_use'] = 'Landed'

df_producer_geo = df_producer_geo.merge(routing_parameters)
df_producer_geo['domestic_demand_total'] = df_producer_geo['Domestic'] * df_producer_geo['domestic_demand']
df_producer_geo['non_domestic_demand_total'] = df_producer_geo['Non Domestic'] * df_producer_geo['non_domestic_demand']
df_producer_geo['hk_domestic_demand_total'] = df_producer_geo['H/M'] * df_producer_geo['hk_demand']
df_producer_geo['demand'] = df_producer_geo['domestic_demand_total'] + df_producer_geo['non_domestic_demand_total'] + df_producer_geo['hk_domestic_demand_total']
df_producer_geo.groupby('land_use')['demand'].sum()
df_producer_geo.groupby('land_use')['total units'].sum()
df_producer_geo['demand'].sum()
df_producer_geo.groupby('land_use')['bin_service_cost'].unique()
df_producer_geo.shape

In [ ]:
m1 = df_visual.plot_keplergl({'Producers': df_producer_geo.drop(columns='geometry')}, height=800, config_path='config/rel_producers_amk.config')
display(m1)

## Select infrastructure

In [ ]:
infrastructure = 'infrastructure/infrastructure_location.csv'
key_pois = pd.read_csv(infrastructure, dtype={'Postal Code': str})
key_pois['u'] = key_pois['arc_u']
key_pois['v'] = key_pois['arc_v']
key_pois['key'] = 0
key_pois['geometry_point'] = key_pois['geometry']
key_pois['geometry'] = key_pois['geometry_arc']
key_pois = process_gdf.process_edges(key_pois)
key_pois['geometry'] = key_pois['geometry_point']
key_pois['Postal Code'] = key_pois['Postal Code'].astype(float).astype(int).astype(str)
key_pois.drop(columns=['arc_index'], inplace=True)
key_pois = key_pois.merge(test_network[['geom_id', 'arc_index']])

Possible depot locations:

In [ ]:
offload = key_pois.loc[(key_pois['type'] != 'depot')]
offload[['description', 'type', 'Postal Code', 'ADDRESS']]

Select depot:

In [ ]:
infrastructure = 'infrastructure/infrastructure_location.csv'

key_depos = key_pois.loc[key_pois['type'] == 'depot']

key_pois_list = list(key_depos['description'].unique())
key_pois_list


description = 'ALBA W&H Tuas depot'

key_depot_select = key_depos.copy()
key_depot_select = key_depot_select.loc[key_depot_select['description'] == description]
depot_arc_index = key_depot_select.iloc[0]['arc_index']
display(key_depot_select[['description', 'type', 'Postal Code', 'ADDRESS']])

Select treatment infrastructure:

In [ ]:
offload = key_pois.loc[key_pois['type'] != 'depot']

descriptions=['KSTP', 'Tuas Incineration Plan', 'Tuas South Incineration Plant']

key_if_select = offload.copy()
key_if_select = key_if_select.loc[key_if_select['description'].isin(descriptions)]
if_arc_index = key_if_select['arc_index'].values
display(key_if_select[['description', 'type', 'Postal Code', 'ADDRESS']])

## Select producers

In [ ]:
key_list = list(df_producer_geo['category'].unique())
key_list


descriptions = ['Landed']

prod_select = df_producer_geo.copy()
prod_select = prod_select.loc[prod_select['category'].isin(descriptions)]
display('Number of producers: {}'.format(prod_select.shape[0]))

## Generate service network

In [ ]:
logging.info('Preparing road network.......\n')

network_info = NetworkExtract(test_network, test_network_directed, nodes, networkh5_file, round_cost=True, key_pois=key_pois)
depot = depot_arc_index
ifs = if_arc_index
req_arcs = network_info.arc_consolidation_standard(prod_select)
network_info.load_required_arcs(req_arcs, merge_network=True)
network_info.set_depot_arc(depot)
network_info.set_if_arcs(ifs)
network_info.loc_in_required_arcs()
network_info.extend_required_inverse_arcs()
network_info.check_main_list()
network_info.load_distance_matrix()
network_info.offload_calculations3D()

network_info.network_gdf(prod_select)

logging.info('Done!')

## Check network connectivity

In [ ]:
d_max = network_info.d.max()
if d_max == np.inf:
    d_i = network_info.d.argmax(axis=0)[0]
    bad_arc = network_info.reqArcList[d_i]
    print(network_info.d[d_i, :])
    print(network_info.d[:, d_i])
    d_i2 = network_info.d[d_i, :].argmax()
    bad_arc2 = network_info.reqArcList[d_i2]
    print(network_info.d[d_i2, :])
    print(network_info.d[:, d_i2])
    test_network.loc[test_network['arc_index'] == bad_arc]
    test_network.loc[test_network['arc_index'] == bad_arc2]

## Set scenario input parameters

In [ ]:
scenario_name = 'Landed refuse'
schedule_select = 'daily'
offload_time = 15 * 60
max_duration = 10.5 * 3600
capacity = 10.5 * 1000
travel_speed = 45 / 3.6
service_speed = 30 / 3.6

demand_increase_factor = {'daily': 1, 'weekly': 7, 'two_one': 2.3}
increase_factor = demand_increase_factor[schedule_select]

logging.info('Uploading scenario inputs...')
prod_select_update = prod_select.copy()
prod_arc_update = prod_select_update.copy()

network_info.producer_demand = prod_arc_update
prod_arc_update = network_info.add_demand_service_cost_to_ars(prod_select_update, service_speed)
_ = pd.DataFrame(prod_arc_update.drop(columns=['arc_index']).sum(numeric_only=True)).astype(int)

network_info.set_travel_speed(travel_speed)
network_info.set_service_cost_and_demand(prod_arc_update)
network_info.set_offload_time(offload_time)
network_info.check_shapes()

network_info.update_cost_matrix()
network_info.update_offload_cost()

network_info.set_vechile_capacity_constraint(capacity)
network_info.set_vehicle_duration_constraint(max_duration)
logging.info('Done')

## Solve problem

In [ ]:
solution_df = solve_store_instance('', 
                                   improve = 'LS', 
                                   write_results=False, 
                                   info=network_info, 
                                   overwrite=True, 
                                   test_solution=True, 
                                   full_output=False, 
                                   tollerance=60, nnFracLS=1, nnFracTS=0.25)
logging.info('Done generating routes')
n_route = solution_df['route'].max() + 1
print(f'Number of routes: {n_route}')

## Display KPIs

In [ ]:
logging.info('Preparing scenario outputs...')

network_info.extend_prop_info()
network_info.add_solution(solution_df, order=False)
network_info.deconstruct_solution()

network_info.vehicle_collection_schedule(schedule=schedule_select)
network_info.extract_wide_offload_totals()

network_info.add_traversal_time()
network_info.add_time_formatted()
network_info.add_constant_duration_time()
_ = network_info.full_producer_report()

route_summary = RouteSummary(network_info)
route_sum_table = route_summary.route_summary()
#route_sum_table['Route cost ($)'] = route_sum_table['Traveling distance (km)'] * 20 + route_sum_table['Route duration (h)'] * 50
#route_sum_table['Disposal cost ($)'] = route_sum_table['Demand collected (ton)'] * 77
#route_sum_table['Capacity utilisation'] = (route_sum_table['Demand collected (ton)'] / (route_sum_table['Offloads'] * (capacity / 1000))).round(2)
#route_sum_table['Time utilisation'] = (route_sum_table['Route duration (h)'] / (max_duration / 3600)).round(2)
#route_sum_table.iloc[-1, -1] = 0
#route_sum_table['# Properties per hour'] = route_sum_table['Units served'] / route_sum_table['Route duration (h)']
#route_sum_table['# Time (seconds) per bin'] = (route_sum_table['Time collecting (h)'] / route_sum_table['Bins collected'] * 3600).astype(int)
#route_sum_table.iloc[-1, -1] = 0
logging.info('Done preparing output')
display(route_sum_table)

## Show routes

In [ ]:
route_plot = network_info.df_solution_full.copy()
route_plot['route'] = route_plot['route'] + 1
smaller_10 = route_plot['route'] < 10
route_plot.loc[smaller_10, 'route'] = 'Route 0' + route_plot.loc[smaller_10]['route'].astype(int).astype(str)
route_plot.loc[~smaller_10, 'route'] = 'Route ' + route_plot.loc[~smaller_10]['route'].astype(int).astype(str)

route_plot['width'] = 0
route_plot.loc[route_plot['activity_type'] == 'collect', 'width'] = 1

route_plot['collection'] = 0
route_plot.loc[route_plot['activity_type'] == 'collect', 'width'] = 1


route_plot = gdf_helpers.create_gdf(route_plot)
route_map = df_visual.plot_keplergl(height=800, data={"routes": route_plot,
                                               "depot": key_depot_select,
                                               "disposal": key_if_select,
                                               "producers": prod_select},
                                    config_path='config/route_complete_rel.cf')

display(route_map)

In [ ]:
#df_visual.save_config(route_map, 'kepler_configs/route_complete_rel.cf', overwrite=True)
map_data = {"routes": route_plot.copy(),
                                                       "depot": key_depot_select.copy(),
                                                       "disposal": key_if_select.copy(),
                                                       "producers": prod_select.copy()}
df_visual.save_html(route_map, data = map_data, path=f'maps/{scenario_name}.html', overwrite=True, aws_host_save=True)

## Save output

In [ ]:
logging.info('Writing outup files...')
r = plot_routes(network_info, key_pois, mapbox_key, customers=prod_select, cust_size={'radiusMinPixels': 0.75, 'radiusMaxPixels': 10, 'get_radius': 2, 'color':'lightgrey'}) # , prod_select
network_info.add_traversal_time()
network_info.add_time_formatted()
network_info.add_constant_duration_time()
_ = network_info.full_producer_report()
network_info.store_results(scenario_name, prod_select, route_sum_table, r)